Importar librerias

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_table("748_20220531_1211_Turnero_F.txt", 
    delimiter=";", encoding = "ISO-8859-1",
    usecols=['Sucursal', 'Proceso', 'Tipo_atencion', 'Fecha_creada', 'Fecha_turno','Estado', 'Hora_desde', 'Hora_hasta', 'Tipo_Doc', 'Numero_doc'],
    dtype={'Sucursal':str, 'Proceso':str, 'Tipo_atencion':str, 'Fecha_creada':object, 'Fecha_turno':object, 'Estado':str, 'Hora_desde':object, 'Hora_hasta':object, 'Tipo_Doc':str, 'Numero_doc':str})

In [ ]:
df['Fecha_creada'] = pd.to_datetime(df['Fecha_creada'], format='%d/%m/%Y %H:%M:%S %p')
df['Fecha_turno'] = pd.to_datetime(df['Fecha_turno'], format='%d/%m/%Y')
df['Duracion_turno'] = (pd.to_datetime(df['Hora_hasta'], format='%H:%M') - pd.to_datetime(df['Hora_desde'], format='%H:%M')).astype('timedelta64[m]')

In [ ]:
df.drop('Hora_hasta', axis=1, inplace=True)

In [ ]:
#df.set_index("Fecha_creada", inplace=True)
#df.sort_index().loc["2022-01-01":]

In [ ]:
#nulls = df.set_index('Fecha_turno').groupby('Estado').resample('M').count().reset_index(name="count")
#nulls = df.set_index('Fecha_turno').groupby('Estado').resample('M').size().reset_index(name='Counts')
#nulls = df.groupby(['Fecha_turno', 'Estado']).size().reset_index(name='Counts')
#nulls = df.set_index('Fecha_turno').resample('M').size().reset_index(name='Counts')
nulls = df.groupby([pd.PeriodIndex(df['Fecha_turno'], freq='M'), 'Estado']).size().reset_index(name='Counts')


In [ ]:
nulls

In [ ]:
# compute null percentage per (id, hour)
nulls = (df.set_index('Fecha_creada').groupby(['Estado', pd.Grouper(freq='M')])['Duracion_turno'].apply(lambda x: x.isnull().mean() * 100))

In [ ]:
nulls

In [ ]:
# pivot into id vs time matrix
#(nulls.pivot(index='Fecha_turno', columns='Estado', values='Count'))
matrix = pd.pivot_table(nulls, values='Counts', index='Estado', columns='Fecha_turno', aggfunc='count')

In [ ]:
import seaborn as sns

In [ ]:
matrix

In [ ]:
# plot time series heatmap
sns.heatmap(matrix, square=True, cmap='magma_r', cbar_kws={'label': 'null (%)'},
            linewidth=1, linecolor='lightgray', clip_on=False, xticklabels=matrix.columns)

In [ ]:
# convert to proper dtypes
df['time'] = pd.to_datetime(df['time'])
df['fuel'] = pd.to_numeric(df['fuel'], errors='coerce')

# compute null percentage per (id, hour)
nulls = (df.set_index('time')
           .groupby(['id', pd.Grouper(freq='H')])['fuel']
           .apply(lambda x: x.isnull().mean() * 100))

# pivot into id vs time matrix
matrix = (nulls.reset_index(name='null (%)')
               .pivot(index='id', columns='time', values='null (%)'))

# plot time series heatmap
sns.heatmap(matrix, square=True, vmin=0, vmax=100, cmap='magma_r', cbar_kws={'label': 'null (%)'},
            linewidth=1, linecolor='lightgray', clip_on=False,
            xticklabels=matrix.columns.strftime('%b %d, %Y\n%H:%M:%S'))